In [25]:
import torch
import esm
import pandas as pd
import pickle
import os
from tqdm import tqdm
import numpy as np

# Set up

First let's install the requirements, if you haven't done that yet.

In [26]:
# !python -m pip install -r requirements.txt

# Embedding generation

We are going to generate [ESM-2](https://github.com/facebookresearch/esm) protein language model embeddings for the heavy chain and average over the whole sequence. Other potential approaches could be to utilise the light chain as well and/or only embed or average over the CDR regions. You could also try other models, such as [ProtBert](https://github.com/nadavbra/protein_bert) or [ProtTrans](https://github.com/agemagician/ProtTrans).

You can download the embeddings we generated here by uncommenting and running the following command.

In [27]:
# !python -m pip install awscli
# !aws s3 sync s3://lemanic-adaptyv-2024/embeddings embeddings --no-sign-request

In [28]:
# BATCH_SIZE = 32  # Make the batch size smaller if you run out of memory

# model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
# batch_converter = alphabet.get_batch_converter()
# device = (
#     "cuda" if torch.cuda.is_available() else "cpu"
# )  # Running this on GPU will be significantly faster
# model = model.to(device)
# model.eval()

# if not os.path.exists("embeddings"):
#     os.makedirs("embeddings")
    
# for csv_file in [
#     "literature_test.csv",
#     "literature_train.csv",
#     "experiment_test.csv",
#     "experiment_train.csv",
# ]:
#     df = pd.read_csv(os.path.join("data", csv_file))
#     name = csv_file[:-4]
#     if not os.path.exists(os.path.join("embeddings", name)):
#         os.makedirs(os.path.join("embeddings", name))
#     data = [(index, row["VHorVHH"]) for index, row in df.iterrows()]
#     data_len = len(data)
#     index_range = list(range(0, data_len, BATCH_SIZE))
#     for i in tqdm(index_range):
#         batch_labels, batch_strs, batch_tokens = batch_converter(
#             data[i : min(data_len, i + BATCH_SIZE)]
#         )
#         batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

#         with torch.no_grad():
#             results = model(
#                 batch_tokens.to(device=device), repr_layers=[33], return_contacts=True
#             )
#         token_representations = results["representations"][33].cpu().numpy()

#         for seq_index, (tokens_len, label) in enumerate(zip(batch_lens, batch_labels)):
#             with open(os.path.join("embeddings", name, f"{label}.pickle"), "wb") as f:
#                 pickle.dump(
#                     token_representations[seq_index, 1 : tokens_len - 1].mean(0), f
#                 )

If you are generating the embeddings, restart the kernel before moving on to the next cells to free up the RAM.

In [29]:
import os
import pickle

import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

# Literature dataset

First let's tackle the dataset that simulates learning on literature, with more positive samples in the training set.

In [30]:
df_train = pd.read_csv("data/literature_train.csv")
X_train = []
y_train = []
for i, row in df_train.iterrows():
    with open(os.path.join("embeddings", "literature_train", f"{i}.pickle"), "rb") as f:
        emb = pickle.load(f)
    X_train.append(emb)
    y_train.append(int(row["Binds"]))
X_train = np.stack(X_train)
y_train = np.array(y_train)

df_test = pd.read_csv("data/literature_test.csv")
X_test = []
y_test = []
for i, row in df_test.iterrows():
    with open(os.path.join("embeddings", "literature_test", f"{i}.pickle"), "rb") as f:
        emb = pickle.load(f)
    X_test.append(emb)
    y_test.append(int(row["Binds"]))
X_test = np.stack(X_test)
y_test = np.array(y_test)

We train a simple random forest classifier on the heavy chain embeddings, with SMOTE resampling to balance the classes.

In [31]:
pca = PCA(n_components=128)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

random_f1 = []
non_random_f1 = []
for seed in range(5):
    clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=seed)
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    non_random_f1.append(f1_score(y_test, predictions))
    random_f1_ = []
    for _ in range(100):
        np.random.shuffle(predictions)
        random_f1_.append(f1_score(y_test, predictions))
    random_f1.append(np.mean(random_f1_))
print(f"Non-random F1: {np.mean(non_random_f1):.2f} ± {np.std(non_random_f1):.2f}")
print(f"Random F1: {np.mean(random_f1):.2f} ± {np.std(random_f1):.2f}")

Non-random F1: 0.35 ± 0.01
Random F1: 0.24 ± 0.00


We can compare the result with a random baseline and see that the model is making meaningful predictions (although they can certainly be improved).

# Experiment dataset

Now let's move on to the experiment simulation dataset, which is more similar to the test set. We will train the same model on the heavy chain embeddings.

In [32]:
df_train = pd.read_csv("data/experiment_train.csv")
X_train = []
y_train = []
for i, row in df_train.iterrows():
    with open(os.path.join("embeddings", "literature_train", f"{i}.pickle"), "rb") as f:
        emb = pickle.load(f)
    X_train.append(emb)
    y_train.append(int(row["Binds"]))
X_train = np.stack(X_train)
y_train = np.array(y_train)

df_test = pd.read_csv("data/experiment_test.csv")
X_test = []
y_test = []
for i, row in df_test.iterrows():
    with open(os.path.join("embeddings", "literature_test", f"{i}.pickle"), "rb") as f:
        emb = pickle.load(f)
    X_test.append(emb)
    y_test.append(int(row["Binds"]))
X_test = np.stack(X_test)
y_test = np.array(y_test)

In [33]:
pca = PCA(n_components=128)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

random_f1 = []
non_random_f1 = []
for seed in range(5):
    clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=seed)
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    non_random_f1.append(f1_score(y_test, predictions))
    random_f1_ = []
    for _ in range(100):
        np.random.shuffle(predictions)
        random_f1_.append(f1_score(y_test, predictions))
    random_f1.append(np.mean(random_f1_))
print(f"Non-random F1: {np.mean(non_random_f1):.2f} ± {np.std(non_random_f1):.2f}")
print(f"Random F1: {np.mean(random_f1):.2f} ± {np.std(random_f1):.2f}")

Non-random F1: 0.27 ± 0.02
Random F1: 0.25 ± 0.01


Here we can see that the same naive method leads to results that are not better than chance. This might be because the embeddings are not capturing the relevant information, or because more sophisticated methods for battling class imbalance are required.